<a id="setup"></a>
# <p style="background-color: blue; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Cleaning & Transformation</p>

<a id="libraries"></a>
# <b><span style='color:lightblue'> Importing Necessary Libraries</span></b>

In [194]:
import pandas as pd
import numpy as np
import os
import sys

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

<a id="load_dataset"></a>
# <b><span style='color:lightblue'> Load Dataset</span></b>

In [195]:
# Loading train df

sys.path.append("..") 

train_df = pd.read_csv('../data/train.csv')

df = train_df

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,...,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8,No
1,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45,No
2,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45,Yes
3,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95,Yes
4,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,...,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,,No


<a id="setup"></a>
# <p style="background-color: #4893D7; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 3 | Handling Naming Conventions</p>
<a id="libraries"></a>

In [196]:
# No extra value gained from retaining phone service within 'No phone service' (contained within other features), so will change to No. Need to locate first.

search_values = ['No internet service', 'No phone service']
matching_columns = []

# Create loop
for col in df.columns:
    if df[col].astype(str).isin(search_values).any():
        matching_columns.append(col)

print("Columns containing service flags:", matching_columns)

Columns containing service flags: ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']


In [197]:
# Creating function

def replace_service_values(df, columns, values_to_replace=None, replacement_value='No'):
    if values_to_replace is None:
        values_to_replace = ['No internet service', 'No phone service']

    for col in columns:
        df.loc[:, col] = df[col].replace(values_to_replace, replacement_value)

    return df


In [198]:
# Applying function

matching_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

df = replace_service_values(df, matching_columns)


In [199]:
# Checking they have been replace by calling first loop function
search_values = ['No internet service', 'No phone service']
matching_columns = []

# Create loop
for col in df.columns:
    if df[col].astype(str).isin(search_values).any():
        matching_columns.append(col)

print("Columns containing service flags:", matching_columns)

Columns containing service flags: []


<a id="setup"></a>
# <p style="background-color: #4893D7; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 4 | Handling Missing Values</p>
<a id="libraries"></a>

In [200]:
# Need to investigate TotalCharges column which we found to be a) non-numeric, and b) include 11 null-like missing values

custom_null_values = [
    '', ' ', '   ', 'NA', 'na', 'Na', 'NaN', 'nan', 'Nan',
    'null', 'NULL', 'Null', 'None', 'none'
]

mask = df.astype(str).isin(custom_null_values)
rows_with_custom_nulls = df[mask.any(axis=1)]
rows_with_custom_nulls



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,...,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,,No
282,1371-DWPAZ,Female,0,Yes,Yes,0,No,No,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
2419,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No,...,No,No,No,No,Two year,No,Mailed check,25.35,,No
2734,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,One year,Yes,Mailed check,19.70,,No
2903,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,19.85,,No
3974,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
5023,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,20.25,,No
5030,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,20.00,,No
5343,4472-LVYGI,Female,0,Yes,Yes,0,No,No,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
5599,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No,...,No,No,No,No,Two year,No,Mailed check,25.75,,No


In [201]:
# Create function to convert to numeric column 

def convert_column_to_numeric(df, column_name):
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    return df


In [202]:
# Apply function

convert_column_to_numeric(df, column_name='TotalCharges')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,...,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.80,No
1,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45,No
2,3797-VTIDR,Male,0,Yes,No,1,No,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45,Yes
3,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95,Yes
4,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,...,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0684-AOSIH,Male,0,Yes,No,1,Yes,No,Fiber optic,Yes,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.00,95.00,Yes
5630,5982-PSMKW,Female,0,Yes,Yes,23,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),91.10,2198.30,No
5631,8044-BGWPI,Male,0,Yes,Yes,12,Yes,No,No,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,21.15,306.05,No
5632,7450-NWRTR,Male,1,No,No,12,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.45,1200.15,Yes


In [203]:
# Confirm function has worked

df['TotalCharges'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 5634 entries, 0 to 5633
Series name: TotalCharges
Non-Null Count  Dtype  
--------------  -----  
5624 non-null   float64
dtypes: float64(1)
memory usage: 44.1 KB


In [204]:
# Locate the nulls

empty_rows = df[df['TotalCharges'].isna()]
empty_rows


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,...,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN,No
282,1371-DWPAZ,Female,0,Yes,Yes,0,No,No,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
2419,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No,...,No,No,No,No,Two year,No,Mailed check,25.35,NaN,No
2734,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,One year,Yes,Mailed check,19.70,NaN,No
2903,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,19.85,NaN,No
3974,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
5023,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,20.25,NaN,No
5030,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No,...,No,No,No,No,Two year,No,Mailed check,20.00,NaN,No
5343,4472-LVYGI,Female,0,Yes,Yes,0,No,No,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
5599,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No,...,No,No,No,No,Two year,No,Mailed check,25.75,NaN,No


In [205]:
# Create function to remove missing values

def remove_missing_values(df,column_name):
    df = df[df[column_name].notna()]
    return df

In [206]:
# Removing rows where Total Charges are null
df = remove_missing_values(df,column_name='TotalCharges')

In [207]:
# Confirm still no true missing values

df.isna().sum().sum()

0

In [208]:
# Confirm still no null-like values

custom_null_values = [
    '', ' ', '   ', 'NA', 'na', 'Na', 'NaN', 'nan', 'Nan',
    'null', 'NULL', 'Null', 'None', 'none'
]

mask = df.astype(str).isin(custom_null_values)
rows_with_custom_nulls = df[mask.any(axis=1)]
rows_with_custom_nulls

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


<a id="setup"></a>
# <p style="background-color: #4893D7; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 5 | Handling Duplicates</p>
<a id="libraries"></a>

In [209]:
# Finding duplicate rows and ordering them by customerID
duplicate_rows = df[df.duplicated(keep=False)]

duplicate_rows_sorted = duplicate_rows.sort_values(by=['customerID'])

duplicate_rows_sorted.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [210]:
# There are no duplicate rows (non-normal, again), so no action here

In [211]:
for col in df.columns:
    num_duplicates = df[col].duplicated().sum()
    if num_duplicates > 0:
        print(f"Column '{col}' has {num_duplicates} duplicate values.")
    else:
        print(f"Column '{col}' has all unique values.")


Column 'customerID' has all unique values.
Column 'gender' has 5622 duplicate values.
Column 'SeniorCitizen' has 5622 duplicate values.
Column 'Partner' has 5622 duplicate values.
Column 'Dependents' has 5622 duplicate values.
Column 'tenure' has 5552 duplicate values.
Column 'PhoneService' has 5622 duplicate values.
Column 'MultipleLines' has 5622 duplicate values.
Column 'InternetService' has 5621 duplicate values.
Column 'OnlineSecurity' has 5622 duplicate values.
Column 'OnlineBackup' has 5622 duplicate values.
Column 'DeviceProtection' has 5622 duplicate values.
Column 'TechSupport' has 5622 duplicate values.
Column 'StreamingTV' has 5622 duplicate values.
Column 'StreamingMovies' has 5622 duplicate values.
Column 'Contract' has 5621 duplicate values.
Column 'PaperlessBilling' has 5622 duplicate values.
Column 'PaymentMethod' has 5620 duplicate values.
Column 'MonthlyCharges' has 4130 duplicate values.
Column 'TotalCharges' has 325 duplicate values.
Column 'Churn' has 5622 duplica

In [212]:
# Importantly, customerID has no duplicate values, all other columns with duplicate values are expected

<a id="setup"></a>
# <p style="background-color: #4893D7; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 6 | Handling Numerical Values</p>
<a id="libraries"></a>

In [213]:
# As identified in EDA, some tenure values are 0 so need to be investigated. Will investogate other numerical values alongside.

numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

for col in numeric_columns:
    print(f"{col} min: {df[col].min()}")
    print(f"{col} max: {df[col].max()}")
    

SeniorCitizen min: 0
SeniorCitizen max: 1
tenure min: 1
tenure max: 72
MonthlyCharges min: 18.25
MonthlyCharges max: 118.6
TotalCharges min: 18.8
TotalCharges max: 8684.8


In [214]:
# Rows have been removed when TotalCharges == 0 rows were removed.
# No other unwanted 0 values (expected in SeniorCitizen as binary)

In [215]:
# Save cleaned .csv 

df.to_csv('../data/cleaned_train_df.csv', index=False)
